In [1]:
%matplotlib inline

# 中文分詞(cntk)

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import pylab
import os
import pickle
import codecs
import glob
import math
import builtins
import string
import cntk as C
from cntk.ops import *
from cntk.layers import *
from cntk.initializer import *
from cntk.logging import *
from cntk.train import *
from cntk.learners import *
from cntk.losses import *
from cntk.metrics import *
from cntk.device import *
import random

# 是否使用GPU
is_gpu = True
if is_gpu:
    try_set_default_device(gpu(0))
else:
    try_set_default_device(cpu())

In [3]:
def str_full_to_half(in_str):
    out_str = []
    for char in in_str:
        inside_code = ord(char)
        if inside_code == 0x3000 or inside_code == 12288 or char==string.whitespace: # 全形空格直接轉換
             out_str.append(' ')
        elif inside_code >= 65281 and inside_code <= 65374:
            inside_code -= 0xfee0
            out_str.append(chr(inside_code))
        else:
            out_str.append(char)
        
    return ''.join(out_str)

In [4]:
as_train=codecs.open('../Data/ex12_train/as_training.utf8',encoding='utf-8-sig').read()
cityu_train=codecs.open('../Data/ex12_train/cityu_training.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_train=as_train.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_train=cityu_train.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data=as_train+'\r\n'+cityu_train #把兩個語料合併
data=data.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data=str_full_to_half(data) #把所有全形轉半形

raw_data_train=data.split('\r\n')#分行

raw_data_train=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_train] #移除分行字元

print(raw_data_train[:20])



['時間|:', '三月|十日|(|星期四|)|上午|十時|。', '地點|:', '學術|活動|中心|一樓|簡報室|。', '主講|:', '民族所|所長|莊英章|先生|。', '講題|:', '閩|、|台|漢人|社會|研究|的|若干|考察|。', '李|院長|於|二月|二十六日|至|三月|十五日|赴|美|訪問|,', '期間|將|與|在|美|院士|商討|院務|,', '與|美國|大學|聯繫|商討|長期|合作|事宜|,', '並|辦理|加州|大學|退休|等|手續|。', '出國|期間|院務|由|羅|副院長|代行|。', '總辦事處|秘書組|主任|戴政|先生|請辭|獲准|,', '所|遺|職務|自|三月|一日|起|由|近代史|研究所|研究員|陶英惠|先生|兼任|。', '植物|研究所|所長|周昌弘|先生|當選|第三世界|科學院|(|The|Third|World|Academy|of|Sciences|,', '簡稱|TWAS|)|院士|。', 'TWAS|係|一九八三年|由|Prof|Adbus|Salam|(|巴基斯坦籍|,', '曾|獲|諾貝爾獎|)|發起|成立|,', '會員|遍佈|63|個|國家|,']


In [5]:
as_test=codecs.open('../Data/ex12_train/as_testing_gold.utf8',encoding='utf-8-sig').read()
cityu_test=codecs.open('../Data/ex12_train/cityu_test_gold.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_test=as_test.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_test=cityu_test.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data_test=as_test+'\r\n'+cityu_test #把兩個語料合併
data_test=data_test.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data_test=str_full_to_half(data_test) #把所有全形轉半形

raw_data_test=data_test.split('\r\n')#分行

raw_data_test=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_test] #移除分行字元

print(raw_data_test[:20])


['許多|社區|長青|學苑|多|開設|有|書法|、|插花|、|土風舞班|,', '文山區|長青|學苑|則|有|個|十分|特別|的|「|英文|歌唱班|」|,', '成員|年齡|均|超過|六十|歲|,', '這|群|白髮蒼蒼|,', '爺爺|、|奶奶級|的|學員|唱起|英文|歌|來|字正腔圓|,', '有模有樣|。', '對|他們|來說|,', '能|與|一|群|志同道合|的|朋友|共同|回味|年少|時期|流行|的|歌曲|,', '才|是|參加|英文|歌唱班|最|大|樂趣|。', '長青|學苑|英文|歌唱班|昨日|在|社教館|舉行|「|英文|懷念|金曲|演唱會|」|,', '曲目|包括|「|大江東去|」|、|「|月河|」|、|「|真善美|」|等|大眾|耳熟能詳|的|英文|歌曲|。', '難得|公開|演唱|,', '這些|有|著|豐富|人生|閱歷|的|學員|絲毫|不|覺得|緊張|怯場|,', '只|見|台|上|唱|得|盡興|,', '台|下|不少|聽眾|也|一時|技癢|跟|著|唱和|起來|。', '長青|學苑|英文|歌唱班|成立|至今|已|兩|年|,', '目前|成員|約|廿五|人|,', '年齡|都|在|六十|歲|以上|,', '其中|以|軍公教|退休|人員|居多|,', '並|有|現任|大學|教授|,']


In [6]:
vocabs=sorted(set(list(''.join(raw_data_train))))
vocabs.remove('|')
vocabs.insert(0,'/Unknow')

print(len(vocabs))
print(vocabs[:100])

6295
['/Unknow', ' ', '!', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '°', '·', '×', 'ˊ', 'ˋ', '˙', 'Α', 'Β', 'Γ', 'Ε', 'Η', 'Ι']


In [7]:
char_to_index=dict((w, i) for i, w in enumerate(vocabs))
index_to_char=dict((i, w) for i, w in enumerate(vocabs))

print({key:char_to_index[key] for key in list(char_to_index.keys())[:100]})

{'駁': 6009, '攤': 2265, '喂': 904, '沒': 2837, '癬': 3585, '汀': 2802, '疏': 3512, '逢': 5393, '杏': 2465, '商': 879, '安': 1398, '孛': 1376, '抨': 2020, '朱': 2456, '協': 686, 'ㄤ': 186, '嵨': 1553, '敗': 2284, '攸': 2272, '康': 1655, '雉': 5814, '傣': 449, '標': 2657, '隰': 5802, '值': 422, '綺': 4121, '嚼': 1031, '們': 402, '答': 3944, '鄂': 5482, '醫': 5530, '冼': 544, '℃': 115, '村': 2467, '停': 432, '孽': 1392, '稷': 3862, '洌': 2881, 'r': 77, '植': 2593, '慟': 1897, '哺': 854, '柏': 2502, '夸': 1221, '捂': 2081, '企': 301, '博': 688, '基': 1123, '敖': 2283, '笪': 3933, '傘': 445, '巢': 1578, '寮': 1445, '臻': 4416, '濮': 3109, '寺': 1450, '忞': 1767, '湎': 2988, '棶': 2588, '右': 742, '箴': 3967, 'b': 61, '線': 4131, '鎔': 5657, '礙': 3785, '嚏': 1019, '叵': 743, '疙': 3515, '梅': 2554, '寨': 1441, '澈': 3086, '壤': 1191, '推': 2126, '壆': 1181, '慰': 1904, '輸': 5322, '春': 2360, '溼': 3015, '骨': 6051, '嗟': 952, '乎': 231, '柺': 2522, '諙': 5029, '窟': 3892, '悟': 1831, '孳': 1387, '都': 5480, '莉': 4547, '擎': 2234, '彪': 1718, '美': 4215, 'i': 68, '決': 2825, 

In [8]:
idx_train=0
idxs_train=np.arange(len(raw_data_train))
np.random.shuffle(idxs_train)

idx_test=0
idxs_test=np.arange(len(raw_data_test))
np.random.shuffle(idxs_test)

def get_next_minibatch(minibatch_size,is_train=True):
    global idx_train,idxs_train,raw_data_train,idx_test,idxs_test,raw_data_test
    features=[]
    labels=[]
    groundtruths=[]
    idx=idx_train
    idxs=idxs_train
    raw_data=raw_data_train
    if is_train==False:
        idx=idx_test
        idxs=idxs_test
        raw_data=raw_data_test
    np.random.shuffle(raw_data)
    
    while len(features)<minibatch_size:
        features_seq=[]
        labels_seq=[]
        groundtruth_seq=[]
        words=raw_data[idxs[idx]].split('|') #轉換成分詞後的詞清單
        #BMES=>[0,1,2,3]
        for word in words:
            for i in range(len(word)):
                feature_arr=np.zeros(len(vocabs),dtype=np.float32)
                label_arr=np.zeros(4,dtype=np.float32)
                #如果在字典中則取出其索引
                if word[i] in char_to_index:
                    feature_arr[char_to_index[word[i]]]=1
                #否則定為未知
                else:
                    feature_arr[char_to_index['/Unknow']]=1
                
                #轉換為BMES
                if len(word)==1 and i==0: #S 自己就是一個單詞
                    label_arr[3]=np.random.choice(np.arange(0.8, 1.2, 0.01))
                elif i==0: #B 是一個詞的開始
                    label_arr[0]=np.random.choice(np.arange(0.8, 1.2, 0.01)) 
                elif i==len(word)-1:  #E 是一個詞的結束
                    label_arr[2]=np.random.choice(np.arange(0.8, 1.2, 0.01))
                else: #M 是一個詞的中間
                    label_arr[1]=np.random.choice(np.arange(0.8, 1.2, 0.01))
                #測試集不進行label smoothing
                if is_train==False:
                    label_arr[label_arr>0]=1
                    
                features_seq.append(feature_arr)
                labels_seq.append(label_arr)
            groundtruth_seq.append(word)
        idx+=1
        
        if idx>len(idxs):
            idx=0
            np.random.shuffle(idxs)
            
        features.append(features_seq)
        labels.append(labels_seq)
        groundtruths.append(groundtruth_seq)
        
    idx_train=idx
    if is_train==False:
        idx_test=idx
    return features,labels,groundtruths
        
x_feature,y_label,ground_truths=get_next_minibatch(2)

print(ground_truths)

[['而', '湯姆斯', '‧'], ['空門', '是', '禪', '的', '修行', '根基', ',']]


In [ ]:
#定義序列軸
input_seq_axis = Axis('inputAxis')

#定義輸入變數
input_sequence = sequence.input_variable(shape=len(vocabs), sequence_axis=input_seq_axis)
label_sequence = sequence.input_variable(shape=4, sequence_axis=input_seq_axis) #BMES四種


#雙向遞迴
def BiRecurrence(fwd, bwd):
    F = C.layers.Recurrence(fwd)
    G = C.layers.Recurrence(bwd, go_backwards=True)
    x = C.placeholder()
    apply_x = C.splice(F(x), G(x))
    return apply_x 

def create_model(num_layers=2,hidden_dim=512):
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(256),
            For(range(num_layers), lambda: 
                Sequential([Stabilizer(), 
                            BiRecurrence(LSTM(hidden_dim//2),LSTM(hidden_dim//2)),
                            BatchNormalization()])),
            Dropout(0.5),
            C.layers.Dense(4)
        ])
 

In [ ]:
learning_rate=0.0001
minibatch_size=32
num_epochs=20

#宣告模型結構
rnn=create_model()
z=rnn(input_sequence)
if os.path.exists("Models/word_segment_cntk.lstm"):
    model=Function.load("Models/word_segment_cntk.lstm")
    z = model(input_sequence)
    print('model recovered!')

#定義進行訓練的損失函數以及錯誤率計算
loss= cross_entropy_with_softmax(z,label_sequence)#+0.1*focal_loss(z,label_sequence)
errs = classification_error(z, label_sequence)

# 列印模型參數
log_number_of_parameters(z);

# 定義訓練器

progress_printer = ProgressPrinter(freq=100, tag='Training', num_epochs=num_epochs)
learner = adam(z.parameters,
                lr=learning_rate_schedule([learning_rate], UnitType.sample, num_epochs),
                momentum=momentum_as_time_constant_schedule([minibatch_size / -math.log(0.95)], epoch_size=num_epochs),
                l2_regularization_weight=5e-4)
trainer = Trainer(z, (loss, 1-errs), learner, progress_printer)
for epoch in range(num_epochs):
    mbs = 0
    progress_printer.update_with_trainer(trainer, with_metric=True)
    num_trained_samples = 0
    while mbs<2000:
        features, labels,ground_truths=get_next_minibatch(minibatch_size)
        #進行訓練
        trainer.train_minibatch({input_sequence: features, label_sequence: labels})
        if mbs%100==0 and mbs>0:
            z.save("Models/word_segment_cntk.lstm")
            features_test, labels_test,ground_truths_test=get_next_minibatch(minibatch_size,False)
            trainer.test_minibatch({input_sequence: features_test, label_sequence: labels_test})
            if mbs%500==0 :

                print('-----測試集驗證--------')
                trainer.summarize_test_progress()
                result=z(features_test)
                for i in range(3):
                    answer='|'.join(ground_truths_test[i])+'|'
                    pred=[]
                    words=list(''.join(ground_truths_test[i]))
                    for j in range(len(words)):
                        word=words[j]
                        onehot=np.argmax(result[i][j])
                        if onehot>=2:
                            pred.append(word+'|')
                        else:
                            pred.append(word)

                    pred=''.join(pred)  
                    print('predict:'+pred)
                    print('answer :'+answer)
                print('-----------------')
            

        if mbs%1000==0 and mbs>0:
            learning_rate*=0.75

        mbs += 1
    #回報每個epoch訓練進度以及相關指標
    trainer.summarize_training_progress()
   
    

model recovered!
Training 4241158 parameters in 21 parameter tensors.
Learning rate per 1 samples: 0.0001
 Minibatch[   1- 100]: loss = -0.249303 * 44659, metric = 92.75% * 44659;
 Minibatch[ 101- 200]: loss = -0.296430 * 44848, metric = 92.44% * 44848;
 Minibatch[ 201- 300]: loss = -0.281151 * 44827, metric = 92.31% * 44827;
 Minibatch[ 301- 400]: loss = -0.268336 * 44810, metric = 92.51% * 44810;
 Minibatch[ 401- 500]: loss = -0.413194 * 45160, metric = 92.34% * 45160;
-----測試集驗證--------
Finished Evaluation [1]: Minibatch[1-5]: metric = 91.28% * 2854;
predict:嚴格|考核|GDP|增長|中|的|環境|成本|,|將|“|環境|資產|增值|”|工作|及|其|成效|作為|考核|政府|行政|績效|的|重要|指標|;|
answer :嚴格|考核|GDP|增長|中|的|環境|成本|,|將|“|環境|資產|增值|”|工作|及其|成效|作為|考核|政府|行政|績效|的|重要|指標|;|
predict:也|早已|拉下|厚重|的|鐵門|,|
answer :也|早已|拉下|厚重|的|鐵門|,|
predict:向|客戶|、|學術界|與|媒體|展示|其|最|新|幾|項|研發|成果|。|
answer :向|客戶|、|學術界|與|媒體|展示|其|最|新|幾|項|研發|成果|。|
-----------------
 Minibatch[ 501- 600]: loss = -0.296743 * 44840, metric = 92.29% * 44840;
 Minibatch[ 601- 700]: loss = -0.29